In [1]:
import pandas as pd
from modules.KIS import public_api, stock
API = public_api.Public_API()
import datetime
import time
import FinanceDataReader as fdr
import pickle
from pprint import pprint

In [2]:
def save_price_info(res, stock_info):
    if res["rt_cd"] == "0":
        stock_code = res["output1"]["stck_shrn_iscd"]
        for j in range(len(res["output2"])):
            try:
                Date = res["output2"][j]["stck_bsop_date"]
                Close = res["output2"][j]["stck_clpr"]
                Open  = res["output2"][j]["stck_oprc"]
                High = res["output2"][j]["stck_hgpr"]
                Low = res["output2"][j]["stck_lwpr"]
                Volume = res["output2"][j]["acml_vol"]
                Amount = res["output2"][j]["acml_tr_pbmn"]
                Change = res["output2"][j]["prdy_vrss"]

                stock_info.loc[len(stock_info)] = [stock_code, Date, Open, High, Low, Close, Volume, Amount, Change]
            except:
                continue
            
    else:
        print(f"Error: {res['rt_cd']}, {res['msg_cd']}", {res['msg1']})
    
    return stock_info
        
    
        
    

    

In [10]:
stock_code = "005930"
stock_info = pd.DataFrame(columns=["Stock_Code","Date", "Open", "High", "Low", "Close", "Volume", "Amount", "Change"])
today = datetime.datetime.today()
str_today = today.strftime("%Y%m%d")
first_date = (datetime.datetime.strptime(str_today, "%Y%m%d") - datetime.timedelta(days=140)).strftime("%Y%m%d")
listing_date = stock.get_listing_date(stock_code)
while datetime.datetime.strptime(str_today, "%Y%m%d") > datetime.datetime.strptime(str(listing_date), "%Y%m%d"):
    print(first_date, str_today)
    res = API.국내주식기간별시세(stock_code, first_date, str_today)
    history_len = len(stock_info)
    stock_info = save_price_info(res, stock_info)
    if stock_info is False:
        break
    if res["rt_cd"] == "0":
        str_today = first_date
        first_date = (datetime.datetime.strptime(str_today, "%Y%m%d") - datetime.timedelta(days=100)).strftime("%Y%m%d")
    else:
        break
    if history_len == len(stock_info):
        break
    

    #time.sleep(0.1)
            
    
    
    
    

    

20221104 20230324
20220727 20221104
20220418 20220727
20220108 20220418
20210930 20220108
20210622 20210930
20210314 20210622
20201204 20210314
20200826 20201204
20200518 20200826
20200208 20200518
20191031 20200208
20190723 20191031
20190414 20190723
20190104 20190414
20180926 20190104
20180618 20180926
20180310 20180618
20171130 20180310
20170822 20171130
20170514 20170822
20170203 20170514
20161026 20170203
20160718 20161026
20160409 20160718
20151231 20160409
20150922 20151231
20150614 20150922
20150306 20150614
20141126 20150306
20140818 20141126
20140510 20140818
20140130 20140510
20131022 20140130
20130714 20131022
20130405 20130714
20121226 20130405
20120917 20121226
20120609 20120917
20120301 20120609
20111122 20120301
20110814 20111122
20110506 20110814
20110126 20110506
20101018 20110126
20100710 20101018
20100401 20100710
20091222 20100401
20090913 20091222
20090605 20090913
20090225 20090605
20081117 20090225
20080809 20081117
20080501 20080809
20080122 20080501
20071014 2

In [11]:
stock_info

,Stock_Code,Date,Open,High,Low,Close,Volume,Amount,Change
0,005930,20230324,62700,63000,62300,62700,11796025,739695869400,400
1,005930,20230323,60600,62300,60600,62300,15381057,946393607300,1200
2,005930,20230322,61000,61200,60500,61100,8978591,546787085426,800
3,005930,20230321,60500,60700,60100,60300,8318514,501917706700,100
4,005930,20230320,61100,61200,60200,60200,9618009,582445819345,-1100
...,...,...,...,...,...,...,...,...,...
8996,005930,19850118,74,74,74,74,549772,1336452,0
8997,005930,19850117,75,75,74,74,832791,2029824,0
8998,005930,19850116,76,76,74,75,319887,787207,-1
8999,005930,19850115,76,76,76,76,873997,2167132,0


In [30]:
stock_info.drop_duplicates(subset=["Date"], inplace=True)
stock_info["Date"] = pd.to_datetime(stock_info["Date"])
stock_info = stock_info.astype({"Open": float, "High": float, "Low": float, "Close": float, "Volume": int, "Amount": int, "Change": float})
stock_info.set_index("Date", inplace=True)
stock_info.sort_index(inplace=True)
stock_info.to_csv("/app/modules/KIS/Data/stock_info.csv")


In [ ]:
fdr_df = fdr.DataReader("005930")
Date_list = []
for i in range(len(fdr_df)):
    Date_list.append(str(fdr_df.index[i])[:10].replace("-", ""))

print(Date_list)

#save pkl file Date_list
import pickle
with open("/app/modules/KIS/Data/Date_list.pkl", "wb") as f:
    pickle.dump(Date_list, f)